<a href="https://www.kaggle.com/code/cozyhn/s3e19-autogluon-timeseries-baseline?scriptVersionId=136662172" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass


In [ ]:
train = pd.read_csv("/kaggle/input/playground-series-s3e19/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s3e19/test.csv")
submission = pd.read_csv("/kaggle/input/playground-series-s3e19/sample_submission.csv")

In [ ]:
train=train.drop("id",axis=1)
test=test.drop("id",axis=1)

In [ ]:
!pip -q install autogluon

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
train.describe(include='all')

In [ ]:
unique_country = train['country'].unique()
unique_country

In [ ]:
unique_store = train['store'].unique()
unique_store

In [ ]:
unique_product = train['product'].unique()
unique_product

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 먼저, 'country', 'store', 'product' 컬럼을 하나의 컬럼으로 합칩니다.
# 이를 위해 각 값들을 문자열로 변환하고, 결합합니다.
train['combined'] = train['country'].astype(str) + "_" + train['store'].astype(str) + "_" + train['product'].astype(str)

# LabelEncoder 객체를 생성합니다.
le = LabelEncoder()

# combined 컬럼의 값들을 학습하고 변환합니다.
train['combined_encoded'] = le.fit_transform(train['combined'])

# 이제 'combined_encoded' 컬럼에는 'country', 'store', 'product'의 각각의 유니크한 조합에 대해 유니크한 숫자가 할당됩니다.


In [ ]:
# 먼저, 'country', 'store', 'product' 컬럼을 하나의 컬럼으로 합칩니다.
# 이를 위해 각 값들을 문자열로 변환하고, 결합합니다.
test['combined'] = test['country'].astype(str) + "_" + test['store'].astype(str) + "_" + test['product'].astype(str)

# LabelEncoder 객체를 생성합니다.
le = LabelEncoder()

# combined 컬럼의 값들을 학습하고 변환합니다.
test['combined_encoded'] = le.fit_transform(test['combined'])

# 이제 'combined_encoded' 컬럼에는 'country', 'store', 'product'의 각각의 유니크한 조합에 대해 유니크한 숫자가 할당됩니다.

In [ ]:
train = train.drop(['combined','country','store','product'], axis=1)

In [ ]:
test = test.drop(['combined','country','store','product'], axis=1)

In [ ]:
train.describe(include='all')

In [ ]:
test.describe(include='all')

In [ ]:
train_data = TimeSeriesDataFrame.from_data_frame(
    train,
    id_column='combined_encoded',
    timestamp_column="date"
)

In [ ]:
train_data.head(20)

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=365,
    target="num_sold",
    eval_metric="sMAPE",
)


In [ ]:
predictor.fit(
    train_data,
    presets="medium_quality",
    time_limit=1200
)

In [ ]:
predictions = predictor.predict(train_data)

In [ ]:
predictions.head(730)

In [ ]:
test = test.rename(columns={'combined_encoded': 'item_id'})
test = test.rename(columns={'date': 'timestamp'})

In [ ]:
test['timestamp'] = pd.to_datetime(test['timestamp'])

In [ ]:
test

In [ ]:
# 'item_id'와 'timestamp'를 기준으로 merge하고, 'mean' 컬럼만 선택
mean_in_order = test.merge(predictions, on=['item_id', 'timestamp'], how='left')['0.6']
mean_in_order

In [ ]:
submission

In [ ]:
submission['num_sold']=mean_in_order

In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)